In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [3]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [4]:
%%capture pwd
!pwd

In [5]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-have-economists-thought-about-the-environment', 300, 500)

# Fig 1

In [6]:
df = pd.read_excel("raw/World_Population_and_oil_prices.xlsx", sheet_name='Population')
df.columns=['year','population','population_growth','population2']
df=df.drop('population2',axis=1)
df['population_growth']/=100.0
df['population']*=1000
df

,year,population,population_growth
0,1870,1.271810e+09,NaN
1,1871,1.280462e+09,0.006802
2,1872,1.289172e+09,0.006802
3,1873,1.297941e+09,0.006802
4,1874,1.306771e+09,0.006802
...,...,...,...
146,2016,7.424485e+09,0.011637
147,2017,7.509410e+09,0.011439
148,2018,7.592476e+09,0.011062
149,2019,7.673345e+09,0.010651


In [7]:
f = "fig1_population"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,population,population_growth
0,1870,1.271810e+09,NaN
1,1871,1.280462e+09,0.006802
2,1872,1.289172e+09,0.006802
3,1873,1.297941e+09,0.006802
4,1874,1.306771e+09,0.006802


In [8]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f"
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-turquiose"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(
    y=alt.Y(
        "population:Q",
        axis=alt.Axis(
            grid=False,
            title="Population",
            titleX=-10,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            orient='right',
            format='~s',
            titleAlign="right",
            labelColor=colors["eco-turquiose"],
            titleColor=colors["eco-turquiose"],
            tickColor=colors["eco-turquiose"],
            domainColor=colors["eco-turquiose"],
        ),
    )
)
line2 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "population_growth:Q",
        axis=alt.Axis(
            grid=False,
            title="Population growth",
            titleX=10,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            orient='left',
            format='.1%',
            titleAlign="left",
            labelColor=colors["eco-dot"],
            titleColor=colors["eco-dot"],
            tickColor=colors["eco-dot"],
            domainColor=colors["eco-dot"],
        ),
    )
)
layer1 = (
    ((line1 + line2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="").resolve_scale(
    y = 'independent'
)
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

npm notice 
npm notice New major version of npm available! 7.7.6 -> 8.2.0
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v8.2.0>
npm notice Run `npm install -g npm@8.2.0` to update!
npm notice 


alt.LayerChart(...)

# Fig 2

In [9]:
df = pd.read_csv("raw/wpd_datasets.csv",skiprows=1)
dfs=[]
for i in range(0,10,2):
    dfs.append(df[df.columns[i:i+2]])
variables=['Resources','Population','Food (per capita)','Industrial output (per capita)','Pollution']
for i in range(len(dfs)):
    dfs[i].columns=['year','value']
    dfs[i]['variable']=variables[i]
df=pd.concat(dfs).dropna().sort_values(by='year')

<ipython-input-9-be40c9579ab7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]['variable']=variables[i]


In [10]:
f = "fig2_world3"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,value,variable
38,1898.120301,0.106579,Population
0,1899.373017,1.000101,Resources
0,1899.920101,0.041509,Industrial output (per capita)
0,1900.138934,0.203318,Food (per capita)
0,1900.686018,0.002213,Pollution


In [11]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect on minutes played",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color='variable:N'
)
layer1 = (
    ((line).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.Chart(...)

# Fig 3

In [780]:
df = pd.read_excel("raw/data.xlsx", skiprows=23, nrows=8, usecols="A:C")
df.columns = ["x", "y", "e"]
df["e1"] = df["y"] - 2 * df["e"]
df["e2"] = df["y"] + 2 * df["e"]
df["x"] = (
    df["x"]
    .replace("to -225", ",> 225")
    .str.replace("-", ",")
    .replace("225 up", ",225 <")
    .str.split(",")
    .str[1]
)

In [781]:
f = "fig3_pass"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,y,e,e1,e2
0,> 225,0.010,0.017,-0.024,0.044
1,225,0.001,0.020,-0.039,0.041
2,150,-0.012,0.018,-0.048,0.024
3,75,0.000,0.000,0.000,0.000
4,75,-0.032,0.020,-0.072,0.008


In [782]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ days before  since infection ➡",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect on pass performance",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(
    size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
)
area = base.mark_area(opacity=0.4, color=colors["eco-turquiose"]).encode(
    y="e1:Q", y2="e2:Q"
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "> 225", "y": 0}, {"x": "225 <", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": "75", "y": -0.16}, {"x": "75", "y": 0.05}]))
    .mark_line(strokeWidth=1, xOffset=25, color=colors["eco-dot"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
layer1 = (
    ((area + line + points + axis1 + axis2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 4

In [783]:
df = pd.read_excel("raw/data.xlsx", skiprows=34, nrows=6, usecols="A:C")
df.columns = ["x", "y", "e"]
df["e1"] = df["y"] - 2 * df["e"]
df["e2"] = df["y"] + 2 * df["e"]

In [784]:
f = "fig4_match_pass"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,y,e,e1,e2
0,01-15,-0.033,0.014,-0.061,-0.005
1,16-30,-0.032,0.014,-0.060,-0.004
2,31-45,-0.065,0.015,-0.095,-0.035
3,46-60,-0.062,0.015,-0.092,-0.032
4,61-75,-0.051,0.017,-0.085,-0.017


In [785]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="minutes player played already",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect on pass performance",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(
    size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
)
top = line.mark_point(
    size=25, shape="line", color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
).encode(y="e2:Q")
area = base.mark_errorbar(
    opacity=0.4, color=colors["eco-turquiose"], ticks=True
).encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
axis1 = (
    alt.Chart(pd.DataFrame([{"x": " 01-15", "y": 0}, {"x": "76-90", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
layer1 = (
    ((top + area + line + points + axis1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 5

In [786]:
df = pd.read_excel("raw/data.xlsx", skiprows=43, nrows=8, usecols="A:C")
df.columns = ["x", "y", "e"]
df["e1"] = df["y"] - 2 * df["e"]
df["e2"] = df["y"] + 2 * df["e"]

In [787]:
f = "fig5_margin"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,y,e,e1,e2
0,Interceptions,-0.027,0.013,-0.053,-0.001
1,Ballrecoveries,-0.022,0.013,-0.048,0.004
2,Short Passes,-0.049,0.014,-0.077,-0.021
3,Long Passes,-0.027,0.013,-0.053,-0.001
4,Pass Accuracy,-0.004,0.004,-0.012,0.004


In [788]:
base = alt.Chart(f5).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=-20,
            labelOffset=10,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(
    size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
)
top = line.mark_point(
    size=25, shape="line", color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
).encode(y="e2:Q")
area = base.mark_errorbar(
    opacity=0.4, color=colors["eco-turquiose"], ticks=True
).encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "Interceptions", "y": 0}, {"x": "Fulltime", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
layer1 = (
    ((top + area + line + points + axis1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 6

## a

In [789]:
df0 = pd.read_excel("raw/data.xlsx", skiprows=23, nrows=8, usecols="A:C")
df0.columns = ["x", "y", "e"]
df0["c"] = "red"
df = pd.read_excel("raw/data.xlsx", skiprows=54, nrows=16, usecols="A:D")
df.columns = ["x", "y", "e", "c"]
df = pd.concat([df0, df], axis=0)
df["e1"] = df["y"] - 2 * df["e"]
df["e2"] = df["y"] + 2 * df["e"]
df["x"] = (
    df["x"]
    .replace("to -225", ",> 225")
    .str.replace("-", ",")
    .replace("225 up", ",225 <")
    .str.split(",")
    .str[1]
)

In [790]:
f = "fig6a_injuries"
f6a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,y,e,c,e1,e2
0,> 225,0.010,0.017,red,-0.024,0.044
1,225,0.001,0.020,red,-0.039,0.041
2,150,-0.012,0.018,red,-0.048,0.024
3,75,0.000,0.000,red,0.000,0.000
4,75,-0.032,0.020,red,-0.072,0.008


In [791]:
base = alt.Chart(f6a).encode(
    color=alt.Color(
        "c:N",
        legend=None,
        scale=alt.Scale(
            range=[colors["eco-purple"], colors["eco-orange"], colors["eco-turquiose"]]
        ),
    ),
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ days before  since infection ➡",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
)
line = base.mark_line().encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect on pass performance",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(size=25,).encode(
    fill=alt.Color(
        "c:N",
        legend=None,
        scale=alt.Scale(
            range=[colors["eco-purple"], colors["eco-orange"], colors["eco-turquiose"]]
        ),
    )
)
area = base.mark_area(opacity=0.2, color=colors["eco-turquiose"]).encode(
    y="e1:Q", y2="e2:Q"
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "> 225", "y": 0}, {"x": "225 <", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": "75", "y": -0.16}, {"x": "75", "y": 0.05}]))
    .mark_line(strokeWidth=1, xOffset=25, color=colors["eco-dot"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
ebar = (
    base.mark_errorbar(
        opacity=0.4, color=colors["eco-turquiose"], ticks=True, rule=False
    )
    .encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
    .transform_filter('datum.x=="225 <"|datum.x=="> 225"')
)
text = (
    line.mark_text(xOffset=12, align="left")
    .encode(text="c2:N")
    .transform_filter('datum.x=="225 <"')
    .transform_calculate(
        c2='datum.c=="red"?"COVID":datum.c=="green"?"short injuries":datum.c=="blue"?"long injuries":datum.c'
    )
)
rule1 = (
    ebar.mark_rule(xOffset=-4, opacity=0.6)
    .transform_filter('datum.x=="225 <"')
    .transform_filter('datum.c=="red"')
)
rule2 = (
    ebar.mark_rule(xOffset=4, opacity=0.6)
    .transform_filter('datum.x=="225 <"')
    .transform_filter('datum.c=="green"')
)
rule3 = (
    ebar.mark_rule(xOffset=8, opacity=0.6)
    .transform_filter('datum.x=="225 <"')
    .transform_filter('datum.c=="blue"')
)
layer1 = (
    (
        (area + ebar + axis1 + axis2 + text + rule1+rule2+rule3+line + points ).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

## b

In [792]:
df0 = pd.read_excel("raw/data.xlsx", skiprows=23, nrows=8, usecols="A:C")
df0.columns = ["x", "y", "e"]
df0["c"] = "red"
df = pd.read_excel("raw/data.xlsx", skiprows=73, nrows=8, usecols="A:D")
df.columns = ["x", "y", "e", "c"]
df = pd.concat([df0, df], axis=0)
df["e1"] = df["y"] - 1 * df["e"]
df["e2"] = df["y"] + 1 * df["e"]
df["x"] = (
    df["x"]
    .replace("to -225", ",> 225")
    .str.replace("-", ",")
    .replace("225 up", ",225 <")
    .str.split(",")
    .str[1]
)

In [793]:
f = "fig6b_infections"
f6b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,y,e,c,e1,e2
0,> 225,0.010,0.017,red,-0.007,0.027
1,225,0.001,0.020,red,-0.019,0.021
2,150,-0.012,0.018,red,-0.030,0.006
3,75,0.000,0.000,red,0.000,0.000
4,75,-0.032,0.020,red,-0.052,-0.012


In [794]:
base = alt.Chart(f6b).encode(
    color=alt.Color(
        "c:N",
        legend=None,
        scale=alt.Scale(
            range=[colors["eco-purple"], colors["eco-turquiose"]]
        ),
    ),
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="⬅ days before  since infection ➡",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
)
line = base.mark_line().encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="COVID effect on pass performance",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.1]),
    )
)
points = line.mark_point(size=25,).encode(
    fill=alt.Color(
        "c:N",
        legend=None,
        scale=alt.Scale(
            range=[colors["eco-purple"], colors["eco-turquiose"]]
        ),
    )
)
area = base.mark_area(opacity=0.2, color=colors["eco-turquiose"]).encode(
    y="e1:Q", y2="e2:Q"
)
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "> 225", "y": 0}, {"x": "225 <", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
axis2 = (
    alt.Chart(pd.DataFrame([{"x": "75", "y": -0.16}, {"x": "75", "y": 0.1}]))
    .mark_line(strokeWidth=1, xOffset=25, color=colors["eco-dot"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
ebar = (
    base.mark_errorbar(
        opacity=0.4, color=colors["eco-turquiose"], ticks=True, rule=False
    )
    .encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
    .transform_filter('datum.x=="225 <"|datum.x=="> 225"')
)
text = (
    line.mark_text(xOffset=12, align="left")
    .encode(text="c2:N")
    .transform_filter('datum.x=="225 <"')
    .transform_calculate(
        c2='datum.c=="red"?"COVID":datum.c=="blue"?"respiratory infections":datum.c'
    )
)
rule1 = (
    ebar.mark_rule(xOffset=4, opacity=0.6)
    .transform_filter('datum.x=="225 <"')
    .transform_filter('datum.c=="red"')
)
rule2 = (
    ebar.mark_rule(xOffset=8, opacity=0.6)
    .transform_filter('datum.x=="225 <"')
    .transform_filter('datum.c=="blue"')
)
layer1 = (
    (
        (area + ebar + axis1 + axis2 + text + rule1+rule2+line + points ).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 7

In [795]:
df = pd.read_excel("raw/data.xlsx", skiprows=84, nrows=5, usecols="A:C")
df.columns = ["x", "y", "e"]
df["e1"] = df["y"] - 2 * df["e"]
df["e2"] = df["y"] + 2 * df["e"]

In [796]:
f = "fig7_quartile"
f7 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x,y,e,e1,e2
0,0,0.000,0.000,0.000,0.000
1,First Quartile,0.004,0.020,-0.036,0.044
2,Second Quartile,-0.025,0.021,-0.067,0.017
3,Third Quartile,-0.017,0.023,-0.063,0.029
4,Fourth Quartile,-0.071,0.035,-0.141,-0.001


In [797]:
base = alt.Chart(f7).encode(
    x=alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="Team minutes of a post-infection player",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
#             labelOffset=10,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="CE effect on team pass performance",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)
points = line.mark_point(
    size=25, color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
)
top = line.mark_point(
    size=25, shape="line", color=colors["eco-turquiose"], fill=colors["eco-turquiose"]
).encode(y="e2:Q")
area = base.mark_errorbar(
    opacity=0.4, color=colors["eco-turquiose"], ticks=True
).encode(y=alt.Y("e1:Q", axis=alt.Axis(title="")), y2="e2:Q")
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "0", "y": 0}, {"x": "Fourth Quartile", "y": 0}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N", sort=[]), y="y:Q")
)
layer1 = (
    ((top + area + line + points + axis1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

Fontconfig error: Cannot load default config file


alt.LayerChart(...)